# Efficient generation of dynamic pulses - Rabi
This script implement a simulated Rabi sweep, as described in the relative [blog post](https://blogs.zhinst.com/andrea/2021/09/08/efficient-generation-of-dynamic-pulses/)


Copyright (C) 2021 Zurich Instruments

This software may be modified and distributed under the terms of the MIT license. See the LICENSE file for details.

In [ ]:
#Define the parameters of the instrument
dataserver_host = 'localhost'    #Hostname or IP address of the dataserer
dev_hd = 'devNNNN'               #Device ID of the HDAWG
awg_core = 0                     #AWG Core (equal to output_channel//2)

In [ ]:
#Imports and initializations
from zhinst.ziPython import ziDAQServer
daq = ziDAQServer(dataserver_host, 8004, 6)

from hdawg_utils import HDAWG_Core
hdawg = HDAWG_Core(daq, dev_hd, awg_core)

## Amplitude Rabi with sequential averaging

In [ ]:
#Waveform paramaters
hdawg.constants.PULSE_WIDTH = 10e-9 #ns
hdawg.constants.SIGMAS_PULSE = 6
hdawg.constants.WAVE_INDEX = 0

#Loop parameters
hdawg.constants.NUM_AVERAGES = 6
hdawg.constants.NUM_AMPLITUDES = 20

#Command table parameters
hdawg.constants.TE_SET_START_AMPLITUDE = 0
hdawg.constants.TE_INCREMENT_AMPLITUDE = 1
hdawg.constants.TE_PULSE = 2

#Readout parameters
hdawg.constants.TRIGGER_LEN = 32
hdawg.constants.READOUT_LEN = 1024

#amplitude paramaters
START_AMPLITUDE = 0.05
INCREMENT_AMPLITUDE = 0.025

#Creation of the command table
ct = [
{
    "index": hdawg.constants.TE_SET_START_AMPLITUDE,
    "waveform": {
        "playZero": True,
        "length": 32
    },
    "amplitude0": {
        "value": START_AMPLITUDE,
        "increment": False
    }
},{
    "index": hdawg.constants.TE_INCREMENT_AMPLITUDE,
    "waveform": {
        "playZero": True,
        "length": 32
    },
    "amplitude0": {
        "value": INCREMENT_AMPLITUDE,
        "increment": True
    }
},{
    "index": hdawg.constants.TE_PULSE,
    "waveform": {
        "index": hdawg.constants.WAVE_INDEX
    }
}
]


seq_definition = """
//Calculations
const PULSE_WIDTH_SAMPLE = PULSE_WIDTH*DEVICE_SAMPLE_RATE;
const PULSE_TOTAL_LEN = ceil(PULSE_WIDTH_SAMPLE*SIGMAS_PULSE*2/16)*16;

//Waveform definition
wave pulse = gauss(PULSE_TOTAL_LEN, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);

//Assign index and outputs
assignWaveIndex(1,pulse, WAVE_INDEX);

//Execution, sequential averaging
executeTableEntry(TE_SET_START_AMPLITUDE);     //Set start amplitude
repeat (NUM_AMPLITUDES) {                      //Amplitude loop
  repeat (NUM_AVERAGES) {                      //Averaging loop
    //resetOscPhase();                         //Reset the oscillator (not really needed for Rabi)
    executeTableEntry(TE_PULSE);               //Play pulse
    playWave(marker(TRIGGER_LEN, 1));          //Readout trigger
    playZero(READOUT_LEN);                     //Readout time
  }
  executeTableEntry(TE_INCREMENT_AMPLITUDE);   //Increment amplitude
}
"""

hdawg.config(seq_definition, ct=ct)
hdawg.oscControl(True)

In [ ]:
hdawg.run()

## Amplitude Rabi with cyclic averaging

In [ ]:
#Waveform paramaters
hdawg.constants.PULSE_WIDTH = 10e-9 #ns
hdawg.constants.SIGMAS_PULSE = 6
hdawg.constants.WAVE_INDEX = 0

#Loop parameters
hdawg.constants.NUM_AVERAGES = 6
hdawg.constants.NUM_AMPLITUDES = 20

#Command table parameters
hdawg.constants.TE_PULSE_START = 0
hdawg.constants.TE_PULSE_INCREMENT = 1

#Readout parameters
hdawg.constants.TRIGGER_LEN = 32
hdawg.constants.READOUT_LEN = 1024

#amplitude paramaters
START_AMPLITUDE = 0.05
INCREMENT_AMPLITUDE = 0.025

#Creation of the command table
ct = [
{
    "index": hdawg.constants.TE_PULSE_START,
    "waveform": {
        "index": hdawg.constants.WAVE_INDEX
    },
    "amplitude0": {
        "value": START_AMPLITUDE,
        "increment": False
    }
},{
    "index": hdawg.constants.TE_PULSE_INCREMENT,
    "waveform": {
        "index": hdawg.constants.WAVE_INDEX
    },
    "amplitude0": {
        "value": INCREMENT_AMPLITUDE,
        "increment": True
    }
}
]


seq_definition = """
//Calculations
const PULSE_WIDTH_SAMPLE = PULSE_WIDTH*DEVICE_SAMPLE_RATE;
const PULSE_TOTAL_LEN = ceil(PULSE_WIDTH_SAMPLE*SIGMAS_PULSE*2/16)*16;

//Waveform definition
wave pulse = gauss(PULSE_TOTAL_LEN, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);

//Assign index and outputs
assignWaveIndex(1,pulse, WAVE_INDEX);

//Execution, cyclic averaging
repeat (NUM_AVERAGES) {                     //Averaging loop
  resetOscPhase();                          //Reset the oscillator (not really needed for Rabi)
  executeTableEntry(TE_PULSE_START);        //Play start pulse
  playWave(marker(TRIGGER_LEN, 1));         //Readout trigger
  playZero(READOUT_LEN);                    //Readout time
  
  repeat (NUM_AMPLITUDES-1) {
    resetOscPhase();                        //Reset the oscillator (not really needed for Rabi)
    executeTableEntry(TE_PULSE_INCREMENT);  //Play pulse and increment amplitude
    playWave(marker(TRIGGER_LEN, 1));       //Readout trigger
    playZero(READOUT_LEN);                  //Readout time
  }
}
"""

hdawg.config(seq_definition, ct=ct)
hdawg.oscControl(True)

In [ ]:
hdawg.run()

## Length Rabi

In [ ]:
#Waveform paramaters
hdawg.constants.PULSE_WIDTH = 10e-9 #ns
hdawg.constants.SIGMAS_PULSE = 6

#Sequence parameters
hdawg.constants.T_START = 64
hdawg.constants.T_END = 10240
hdawg.constants.T_STEP = 27

#Readout parameters
hdawg.constants.TRIGGER_LEN = 32
hdawg.constants.READOUT_LEN = 1024

#amplitude paramaters
hdawg.constants.AMPLITUDE = 1.0

#Creation of the command table
ct = []
for i in range(16):
    entry = {
        "index": i,
        "waveform": {
            "index": i
        }
    }
    ct.append(entry)

seq_definition = """
//Calculations
const PULSE_WIDTH_SAMPLE = PULSE_WIDTH*DEVICE_SAMPLE_RATE;
const PULSE_TOTAL_LEN = ceil(PULSE_WIDTH_SAMPLE*SIGMAS_PULSE*2/16)*16;

//Waveform definition
wave pulse = gauss(PULSE_TOTAL_LEN, AMPLITUDE, PULSE_TOTAL_LEN/2, PULSE_WIDTH_SAMPLE);
wave w_rise = cut(pulse, 0, PULSE_TOTAL_LEN/2-1);
wave w_fall = cut(pulse, PULSE_TOTAL_LEN/2, PULSE_TOTAL_LEN-1);

// +8 samples added on both of the sides
// due to limitation in LabOne 21.08
w_rise = join(zeros(8),w_rise,rect(8, AMPLITUDE));

//Create shifted fall waveforms
cvar j;
for (j = 0; j < 16; j++) {
  //Create the j-samples shifted waveform
  // +8 samples added on both of the sides
  // due to limitation in LabOne 21.08
  wave w_shifted = join(rect(j+8, AMPLITUDE), w_fall, zeros(16-j+8));
  //Assign index j to the waveform
  assignWaveIndex(1, w_shifted, j);
}


//Execution, no averaging
var t = T_START;
var t_fine, coarse;
do {
    // remove added samples due to limitation
    // in LabOne 21.08
    t -= 16;

    t_fine = t & 0xF;                    //The fine shift is the four least significant bits
    coarse = t & -0x10;                  //Check if coarse delay is needed (boolean). Equivalent to (t >= 16)

    resetOscPhase();                     //Reset the oscillator

    playWave(1,w_rise);                  //Play rising edge
    if(coarse)
      playZero(t);                       //keep playing last value (needs awgs/n/outputs/*/hold = True)
    executeTableEntry(t_fine);           //Play falling edge, with fine shift
    
    playWave(marker(TRIGGER_LEN, 1));    //Readout trigger
    playZero(READOUT_LEN);               //Readout time

    t += T_STEP;                         //Increase wait time
} while (t < T_END);                     //Loop until the end
"""

hdawg.config(seq_definition, ct=ct)
hdawg.setHold(True)

In [ ]:
hdawg.run()